In [ ]:
import dlt
from pyspark.sql import functions as F
import pyspark.sql.types as T

**BI GOLD Table**

In [ ]:
@dlt.table(
    comment="Gold layer: curated weather data for BI consumption"
)

def gold_bi():
    silver_df = dlt.readStream("silver_table")

    df_bi = (silver_df.select(
            # ---------------------------
            # 1. Location / Entity
            # ---------------------------
            F.col("city_id"),
            F.col("name").alias("city_name"),
            F.col("country").alias("country_code"),

            # ---------------------------
            # 2. Time (Local + UTC)
            # ---------------------------
            F.col("local_time").alias("observation_time_local"),
            F.col("datetime").alias("observation_time_utc"),

            # ---------------------------
            # 3. Temperature (explicit units)
            # ---------------------------
            F.round("temperature", 1).alias("temperature_c"),
            F.round("feels_like", 1).alias("feels_like_c"),
            F.round("temperature_min", 1).alias("temp_min_c"),
            F.round("temperature_max", 1).alias("temp_max_c"),

            # ---------------------------
            # 4. Atmosphere
            # ---------------------------
            F.col("humidity").alias("humidity_pct"),
            F.col("pressure").alias("pressure_hpa"),
            F.col("clouds_all").alias("cloud_cover_pct"),

            # ---------------------------
            # 5. Conditions (human readable)
            # ---------------------------
            F.col("weather_main").alias("condition_category"),
            F.initcap("weather_description").alias("condition_text"),
            F.col("weather_icon"),

            # ---------------------------
            # 6. Wind
            # ---------------------------
            F.round("windspeed", 1).alias("wind_speed_ms"),
            F.col("wind_deg").alias("wind_direction_deg"),
            F.round("wind_gust", 1).alias("wind_gust_ms"),

            # ---------------------------
            # 7. Rain (clean nulls)
            # ---------------------------
            F.coalesce("rain_1h", F.lit(0.0)).alias("rain_mm")
        )
    )
    
    return df_bi 
